In [10]:
import re
import hashlib
import numpy as np

def preprocess_text(text):
    """ Entfernt Sonderzeichen und führt eine einfache Textbereinigung durch. """
    text = re.sub(r'[^\w\s]', '', text).lower()
    return text

def create_shingles(text, k=3):
    """ Erstellt Shingles (n-Gramme) der Länge k aus dem Text. """
    words = text.split()
    for i in range(len(words) - k + 1):
        yield ' '.join(words[i:i + k])

def minhash(shingles, num_perm=128):
    """ Erstellt eine MinHash-Signatur für ein Set von Shingles. """
    signatures = np.full(num_perm, float('inf'))
    for shingle in shingles:
        for i in range(num_perm):
            hash_val = int(hashlib.sha256(f'{shingle}{i}'.encode()).hexdigest(), 16) % (2**32 - 1)
            signatures[i] = min(signatures[i], hash_val)
    return signatures

def jaccard_similarity(set1, set2):
    """ Berechnet die Jaccard-Ähnlichkeit zwischen zwei MinHash-Signaturen. """
    return np.sum(set1 == set2) / len(set1)

# Lese die Datei und erstelle MinHash für jeden Artikel
file_path = '/Users/furkan/Downloads/articles-2.txt'
articles_minhash = {}

with open(file_path, 'r', encoding='utf-8') as file:
    content = file.readlines()

for line in content:
    parts = line.split(' ', 1)
    if len(parts) == 2:
        article_id, article_text = parts[0], parts[1]
        processed_text = preprocess_text(article_text)
        shingles = set(create_shingles(processed_text))
        minhash_signature = minhash(shingles)
        articles_minhash[article_id] = minhash_signature

# Suche nach ähnlichen Artikel-Paaren und berechne die Jaccard-Ähnlichkeit
plagiarism_candidates = []
article_ids = list(articles_minhash.keys())

for i in range(len(article_ids)):
    for j in range(i + 1, len(article_ids)):
        id1, id2 = article_ids[i], article_ids[j]
        similarity = jaccard_similarity(articles_minhash[id1], articles_minhash[id2])
        if similarity > 0.5:  # Schwellenwert für die Ähnlichkeit
            plagiarism_candidates.append((id1, id2, similarity))

# Ergebnisse ausgeben
for pair in plagiarism_candidates:
    print(f"Artikel {pair[0]} und {pair[1]} ähneln sich mit einer Ähnlichkeit von {pair[2]*100:.2f}%.")


Artikel t980 und t2023 ähneln sich mit einer Ähnlichkeit von 99.22%.
Artikel t1088 und t5015 ähneln sich mit einer Ähnlichkeit von 97.66%.
Artikel t1297 und t4638 ähneln sich mit einer Ähnlichkeit von 98.44%.
Artikel t1768 und t5248 ähneln sich mit einer Ähnlichkeit von 98.44%.
Artikel t1952 und t3495 ähneln sich mit einer Ähnlichkeit von 97.66%.
Artikel t2535 und t8642 ähneln sich mit einer Ähnlichkeit von 97.66%.
Artikel t2839 und t9303 ähneln sich mit einer Ähnlichkeit von 98.44%.
Artikel t2957 und t7111 ähneln sich mit einer Ähnlichkeit von 95.31%.
Artikel t3268 und t7998 ähneln sich mit einer Ähnlichkeit von 98.44%.
Artikel t3466 und t7563 ähneln sich mit einer Ähnlichkeit von 97.66%.
